# Library of functions for the AFM-SINDy Algorithm

### List of dependencies

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.ticker import ScalarFormatter
from matplotlib.ticker import MaxNLocator
from matplotlib.ticker import MultipleLocator
from matplotlib.ticker import FormatStrFormatter
from IPython.display import Image
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm
import numpy as np
from scipy.integrate import solve_ivp
from scipy.interpolate import CubicSpline
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge
from sklearn.linear_model import OrthogonalMatchingPursuit
from sklearn.linear_model import Lars
from sklearn.linear_model import LassoLars
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from pysindy.utils import lorenz, lorenz_control, enzyme
import pysindy as ps
import cvxpy
import pandas as pd
from datetime import datetime
import glob
import dill
import os
import re
import copy

## Miscellaneous Functions

In [ ]:
def set_plot_style(style_num = 17):
    # Version 2.0: This function sets the style of the images that are generated by matplotlib. 
    # New in Version 2.0: Added functionality to put back the plotting style to 'default' when styly_num = 'default'.
    plt.style.use('default')
    plt.style.use(plt.style.available[style_num])

    if style_num == 'default': # New in version 2.0
        plt.style.use('default')

def convert_to_table(equation_string):
    #Version 2.0: This function gets the string of an equation found by SINDy and it splits the string in the different terms of the equation. 
    #             Whenever the function sees a + that is followed by blank spaces like " + ", then it splits the string to generate a term. 
    #             However, whereever the functions sees a + without a space like in "(1+x)^-8", this is not splitted, as is considered as a single term.

    #Added in version 2: The function now can distinguish terms like "(1+x)^-8" as it does not split strings where the + is immidiatly
    #                    followed by strings different that a blank space.

    equation_terms = equation_string
    separated_terms = re.split(r' \+ ', equation_terms)  # Split on space-surrounded plus signs

    # Split each term into components based on spaces, then filter out any empty strings
    term_components = [term.split(' ') for term in separated_terms]
    cleaned_components = [[component for component in components if component] for components in term_components]

    # Create a dictionary to hold the coefficient and variable parts of each term
    coefficients_variables_dict = {}
    for term in cleaned_components:
        variable_part = '*'.join(term[1:])  # Join all parts except the coefficient with '*'
        coefficients_variables_dict[variable_part] = float(term[0])  # Convert coefficient to float and add to dict

    # Convert the dictionary to a pandas Series and return
    equation_series = pd.Series(coefficients_variables_dict)
    return equation_series

def score_tables(table_found, table_ref):
    #Version 1: This function analyzes an equation that has been found by SINDy and that has been splitted in a table using convert_to_table().
    #           The function compares the found equation with the reference equation to verify that the found equation has exacly the same terms
    #           as the reference equation. If both have the same term, then the function return the string "correct". 
    #           Important: The coefficients are not being checked by this function, as these are found by the .fit() method of SINDy.

    # Parameters: "table_found": Is the found equation in its table form.
    #             "table_ref": Is the equation used as a reference. This must have the same terms that the objective equation that SINDy is trying to find. 
    
    for var, val in table_found.items():
        if var not in table_ref:
            return('not correct')
    for var, val in table_ref.items():
        if var not in table_found:
            return('not correct')
    return ('correct')
    
def is_found_eq_same_as_ref_eq(equation, input_dict, should_print = False):
    #Version 1: This function uses the convert_to_table() and the score_tables() to conveniently show if the equation used as reference,
    #           matches the equation found by SINDy. If it matches, it return the string "correct"
    #Parameters: "equation": Is the string of one single equation that SINDy has found. 
    #            "input_dict": Is the referece equation that one wants to find written in a dictionary. 
    #Example: If the equation is  (y)' = -0.995 x + -0.298 y + -0.995 x^3 + 0.199 cos(1*phase), then input_dict2 = {'y': -0.298, 'x':-1, 'x^3': -0.995, 'cos(1*t)': 0.199}. 

    tab = convert_to_table(equation)
    table_ref = pd.Series(input_dict)
    result = score_tables(tab, table_ref)

    if should_print:
        print('Equation used as reference has:')
        print(table_ref)
        print('Temp equation found by SINDy has:')
        print(tab)
        print(' ')
        print('Therefore, the current found equation is: ' + result)
    return result 

## Functions to generate data

In [ ]:
def generate_different_trajectories(dynamical_system, num_trajectories, t_train, init_cond_range, DOF, noise_level, method = 'RK45', parameters=None, noisy_trajectories=False, add_ivp_time_array=False):
    # Version 4.0: This function takes a defined dynamical system and it generates multiple trajectories of synthetic data that starts
    # from different initial conditions.
    # Added in version 2: This function now creates a solution_ivp varible to later split x_train = solution_ivp.y.T & t_train = solution_ivp.t
    # Added in version 3: Now the function can generate different random init_cond within a range. Now the ranges can be written separately for disp, vel and phase.
    # Added in version 4: Now the function also stores every random initial condition used to generate the different trajectories. Also progress bar was added.

    t_train_span = (t_train[0], t_train[-1])
    trajectories_list = []
    init_cond_list = []
    for trajectory in tqdm(range(num_trajectories), desc="Generating Trajectories"):     # For loop with "tqdm" to add a progress bar
        init_cond_disp = np.random.uniform(init_cond_range[0][0], init_cond_range[0][1])
        init_cond_vel = np.random.uniform(init_cond_range[1][0], init_cond_range[1][1])
        init_cond_phase = np.random.uniform(init_cond_range[2][0], init_cond_range[2][1])

        if DOF == 1:
            init_cond = np.array([init_cond_disp])
        elif DOF == 2:
            init_cond = np.array([init_cond_disp, init_cond_vel])
        elif DOF == 3:
            init_cond = np.array([init_cond_disp, init_cond_vel, init_cond_phase])

        init_cond_list.append(init_cond)

        if parameters is None:
            solution_ivp = solve_ivp(dynamical_system, t_train_span, init_cond, t_eval=t_train, rtol=1e-6, atol=1e-8, method=method, dense_output=True)
        else:
            solution_ivp = solve_ivp(dynamical_system, t_train_span, init_cond, t_eval=t_train, args=parameters, rtol=1e-6, atol=1e-8, method=method, dense_output=True)
        
        x_train = solution_ivp.y.T
        t_train_ivp = solution_ivp.t

        if noisy_trajectories:
            x_train = add_gaussian_noise_fixed_std(x_train, noise_level)
        
        if add_ivp_time_array:
            x_train = np.vstack((x_train[:,0], x_train[:,1], t_train)).T

        trajectories_list.append(x_train)

    return trajectories_list, init_cond_list

In [ ]:
def add_gaussian_noise(x_train, noise_level):
    # Version 2.0: This function adds Gaussian noise to the training data.
    # Parameters: x_train: The original training data & noise_level: The desired noise level as a percentage.    
    # Returns: A new dataset with added Gaussian noise.
    # New in Version 2: Adds Gaussian noise only to displacement and velocity (columns 0 and 1)
    
    x_noisy = x_train.copy()
    std_per_feature = np.std(x_train[:, :2], axis=0)  # Only displacement and velocity
    noise_std = std_per_feature * (noise_level / 100.0)
    noise = np.random.normal(0, noise_std, (x_train.shape[0], 2))
    
    x_noisy[:, :2] += noise  # Apply noise only to first two columns
    return x_noisy

def add_gaussian_noise_fixed_std(x_train, std=1e-5):
    # Version 1: Adds Gaussian noise with mean = 0 and fixed std to only displacement and velocity
    # Parameters: x_train: The original training data
    x_noisy = x_train.copy()
    noise = np.random.normal(loc=0.0, scale=std, size=(x_train.shape[0], 2))
    x_noisy[:, :2] += noise  # Only add noise to columns 0 and 1
    return x_noisy

### Functions to treat the data with units

In [ ]:
def get_true_AFM_displacement(afm_disp_array, eta_star, tau_array, omega_0):

    true_displacement = afm_disp_array*eta_star
    true_time = tau_array/omega_0

    return true_displacement, true_time

def get_true_AFM_velocity(afm_vel_array, eta_star, tau_array, omega_0):

    true_velocity = afm_vel_array*(eta_star*omega_0)
    true_time = tau_array/omega_0

    return true_velocity, true_time

def get_true_AFM_Tip_Sample_Force(afm_force_array, eta_star, tau_array, omega_0, cant_density, cant_width, cant_thickness, cant_length):

    true_tip_sample_force = afm_force_array * ((cant_density * cant_width * cant_thickness * ((eta_star * 1e-9) ** 3) * (omega_0 ** 2)) / cant_length)

    true_time = tau_array/omega_0

    return true_tip_sample_force, true_time

def calculate_AFM_DMT_Force(eta_1, a0, C1, C2):
#   Version 1: This functions simulates the F_ts of a DMT model when a0, C1 and C2 are known. 
#              This function it is ment to be used with the function calculate_true_F_ts_DMT()    
    
    if 1-eta_1 <= a0: 
        F_ts_temp = (C1/(a0**2)) + C2* (a0-(1 - eta_1))**1.5 #Repulsive regime
    else: 
        F_ts_temp = (C1/(1-eta_1)**(2))  #Attractive regime Force

    return F_ts_temp 

def calculate_true_F_ts_DMT(sim_disp_data, a0, C1, C2):
    #Version 1.0: This function takes the array of simulated data and calculates the Tip-Sample Force using the DMT model.

    #Parameters: - sim_disp_data: Synthetic data generated with the function AFM_w_DMT().
    #                             It can be either a single measurement in an np.array or multiple trajectories in a list.
    #            - C1, C2 & a0: Float numbers containing the parameters C1, C2 and a0 of DMT model.
    if isinstance(sim_disp_data, np.ndarray):
        F_ts_true_array = np.zeros_like(sim_disp_data[:,0],dtype=float)
        for i in range(len(sim_disp_data)):
            x_elem = sim_disp_data[i, 0]
            F_ts_temp = calculate_AFM_DMT_Force(eta_1=x_elem, a0=a0, C1=C1, C2=C2)
            F_ts_true_array[i] = F_ts_temp
        return F_ts_true_array 

    if isinstance(sim_disp_data, list):
        F_ts_true_list = []
        for trajectory in sim_disp_data:
            F_ts_true_array = np.zeros_like(trajectory[:,0], dtype=float)
            for i in range(len(trajectory)):
                x_elem = trajectory[i, 0]
                F_ts_temp = calculate_AFM_DMT_Force(eta_1=x_elem, a0=a0, C1=C1, C2=C2)
                F_ts_true_array[i] = F_ts_temp
            F_ts_true_list.append(F_ts_true_array.copy())

        return F_ts_true_list
    
def calculate_AFM_DMT_viscoelas_damp_Force(eta_1, eta_2, a0, C1, C2, C3):
#   Version 1: This functions simulates the F_ts of a DMT model when a0, C1, C2 and C3 are known. 
#              This function it is ment to be used with the function calculate_true_F_ts_DMT_viscoelast_damp()
    
    if 1-eta_1 <= a0: 
        F_ts_temp = (C1/(a0**2)) + C2* (a0-(1 - eta_1))**1.5 - C3*(((a0-(1 - eta_1))**(0.5))*eta_2) #Repulsive regime
    else: 
        F_ts_temp = (C1/(1-eta_1)**(2))  #Attractive regime Force

    return F_ts_temp 

def calculate_true_F_ts_DMT_viscoelast_damp(sim_disp_data, a0, C1,C2,C3):
    #Version 1.0: This function takes the array of simulated data and calculates the Tip-Sample Force using the DMT model.

    #Parameters: - sim_disp_data: Synthetic data generated with the function AFM_w_Lennard_Jones().
    #                             It can be either a single measurement in an np.array or multiple trajectories in a list.
    #            - C1, C2 & a0: Float numbers containing the parameters C1, C2 and a0 of DMT model with Kelvin Voigt
    if isinstance(sim_disp_data, np.ndarray):
        F_ts_true_array = np.zeros_like(sim_disp_data[:,0],dtype=float)
        for i in range(len(sim_disp_data)):
            x_elem = sim_disp_data[i, 0]
            y_elem = sim_disp_data[i, 1]
            F_ts_temp = calculate_AFM_DMT_viscoelas_damp_Force(eta_1=x_elem, eta_2=y_elem, a0=a0, C1=C1, C2=C2, C3=C3)
            F_ts_true_array[i] = F_ts_temp
        return F_ts_true_array 

    if isinstance(sim_disp_data, list):
        F_ts_true_list = []
        for trajectory in sim_disp_data:
            F_ts_true_array = np.zeros_like(trajectory[:,0], dtype=float)
            for i in range(len(trajectory)):
                x_elem = trajectory[i, 0]
                y_elem = trajectory[i, 1]  # This line was missing
                F_ts_temp = calculate_AFM_DMT_viscoelas_damp_Force(
                    eta_1=x_elem, eta_2=y_elem, a0=a0, C1=C1, C2=C2, C3=C3)
                F_ts_true_array[i] = F_ts_temp
            F_ts_true_list.append(F_ts_true_array.copy())  # .copy() to avoid overwriting
        return F_ts_true_list

## Functions to generate candidate functions

Here functions are being defined to later be used and called by the function that generates the library itself. 

In [ ]:
def make_sine(f):
    return lambda x: np.sin(f*x)

def make_sine_name(f):
    return lambda x: f"sin({f}*{x})"
#-----------------------------------
def make_cosine(f):
    return lambda x: np.cos(f*x)

def make_cosine_name(f):
    return lambda x: f"cos({f}*{x})"
#-----------------------------------
def make_poly(degree):
    if degree == 1:
        return lambda x: x
    else:
        return lambda x: x**degree

def make_poly_name(degree):
    if degree == 1:
        return lambda x: x
    else:
        return lambda x: x+'^'+str(degree)
#-----------------------------------    
def make_poly_comb(DOF): 
    if DOF == 2:
        return lambda x,y: x*y
    if DOF == 3:
        return lambda x,y,z: x*y*z

def make_poly_comb_name(DOF):
    if DOF == 2:
        return lambda x,y: x+y
    if DOF == 3:
        return lambda x,y,z: x+y+z
#-----------------------------------
def make_poly_frac(degree):
    return lambda x: x**(-degree)

def make_poly_frac_name(degree):
    return lambda x: x+'^'+str(-degree)
#-----------------------------------
def make_denom_cosine(f, degree):
    return lambda x: (np.cos(f*x))**(-degree)

def make_denom_cosine_name(f, degree):
    return lambda x: f"cos({f}*{x})"+'^'+str(-degree)
#-----------------------------------
def make_denom_sine(f, degree):
    return lambda x: (np.sin(f*x))**(-degree)

def make_denom_sine_name(f, degree):
    return lambda x: f"sin({f}*{x})"+'^'+str(-degree)
#-----------------------------------
def make_AFM_LJ_func(degree): # This function is to generate the Lennard Jones term (1-x)^-(degree)
    return lambda x: (1-x)**(-degree)

def make_AFM_LJ_func_name(degree):
    return lambda x:f"({1}-{x})"+'^'+str(-degree)
#-----------------------------------
def make_1_over_eta_func(degree): # This function is to generate (eta)^-(degree)
    return lambda x: (x)**(-degree)

def make_1_over_eta_func_name(degree):
    return lambda x:f"({x})"+'^'+str(-degree)
#-----------------------------------
def make_AFM_LJ_pos_degree_func(degree):   #This function is to generate the term z^degree =  (1-x)^(degree)
    return lambda x: (1-x)**(degree)

def make_AFM_LJ_pos_degree_name(degree):
    return lambda x: f"({1}-{x})"+'^'+str(degree)
#-----------------------------------

def make_AFM_LJ_damp_func(degree, degree_2): #This function is to generate the LJ term ((1-x))^(0.degree))*y with a damping.
    return lambda x,y: ((1 - x)**(degree))*(y**degree_2) 

def make_AFM_LJ_damp_name(degree, degree_2):
    if degree == 1 and degree_2 == 1:
        return lambda x,y: f"(({1}-{x})"+f")*{y}"
    elif degree == 1:
        return lambda x,y: f"(({1}-{x})"+f")*{y}"+'^'+str(degree_2) 
    elif degree_2 == 1:
        return lambda x,y: f"(({1}-{x})"+'^'+str(degree)+f")*{y}"
    else:
        return lambda x,y: f"(({1}-{x})"+'^'+str(degree)+f")*{y}"+'^'+str(degree_2) 
#-----------------------------------

def make_exp_damping_func(z_b, degree):
    return lambda x,y: (y**degree)*(np.exp((1-x)/(z_b)))

def make_exp_damping_name(z_b, degree):
    if degree == 1:
        return lambda x,y: f"{y}"+"*exp"+f"({1}-{x}/{z_b})"
    else:
        return lambda x,y: f"{y}"+'^'+str(degree)+"*exp"+f"({1}-{x}/{z_b})"   
#-----------------------------------
def vdw(x,a0,degree):

    cand_func_array=np.zeros_like(x,dtype=float)
    for ii in range(len(x)):
        if 1 - x[ii]  < a0:
            cand_func_array[ii]=0
        else:
            cand_func_array[ii]=(1-x[ii])**(-degree)
    return cand_func_array

def make_AFM_DMT_att_func(x, a0, degree): #This function is to generate the DMT term (1-x))^-(degree)
    return lambda x: vdw(x,a0,degree)

def make_AFM_DMT_att_name(a0, degree):
    return lambda x: f"({1}-{x})"+'^'+str(-degree) 
#-----------------------------------

def dmt_rep(x,a0,degree):

    cand_func_array=np.zeros_like(x,dtype=float)
    for ii in range(len(x)):
        if 1 - x[ii]  < a0:
            cand_func_array[ii]=np.power(a0 - 1 + x[ii], degree)      
        else:
            cand_func_array[ii]= 0
    return cand_func_array  

def make_AFM_DMT_rep_func(x, a0, degree): #This function is to generate the DMT term (a0-(1-x))^-(degree)
    return lambda x: dmt_rep(x,a0,degree)

def make_AFM_DMT_rep_name(a0, degree):
    return lambda x:  f"({a0}-{1}+{x})" +'^'+str(degree) 
#-----------------------------------

def DMT_rep_viscoel_damp(x, y, a0, degree, degree_2):
#   Version 2: This function generates the discontinous function for the viscoelastic term in a DMT model 
#              of the shape ((a0-1-e1)^degree)*(e2^degree_2).
#   New in Version 2: added the funcitonality for 'y' to have a different exponent than 1 by introducing
#                     the parameter degree_2
#   Parameters : a_0 is the intermolecular distance value
#                degree is the exponent that goes with 'x' variable.
#                degree_2 is the exponent that goes with 'y' variable.

    cand_func_array=np.zeros_like(x,dtype=float)
    for ii in range(len(x)):
        if 1 - x[ii]  < a0:
            cand_func_array[ii]=((a0 - 1 + x[ii])**(degree))*(y[ii]**degree_2) #here before it was hardcoded so that degree was always 0.5
        else:
            cand_func_array[ii]= 0
    return cand_func_array  

def make_AFM_DMT_viscoel_damp_func(x, y, a0, degree, degree_2):                     
    return lambda x,y: DMT_rep_viscoel_damp(x, y, a0, degree, degree_2)

def make_AFM_DMT_viscoel_damp_name(a0, degree, degree_2):
    if degree == 1 and degree_2 == 1:
        return lambda x,y: f"(({a0}-1+{x})"+f")*{y}"
    elif degree == 1:
        return lambda x,y: f"(({a0}-1+{x})"+f")*{y}"+'^'+str(degree_2) 
    elif degree_2 == 1:
        return lambda x,y: f"(({a0}-1+{x})"+'^'+str(degree)+f")*{y}"
    else:
        return lambda x,y: f"(({a0}-1+{x})"+'^'+str(degree)+f")*{y}"+'^'+str(degree_2) 

In [ ]:
def to_ranges(value):
#     Version 2.0: To be used in the function create_candidate_func_w_func_names(). 
#     It Converts an input value to a set of integers (handles both single integers and tuples/ranges)
#     The idea is to be able to generate only the powers of a candidate function that are needed. In a polynomial library for instance
#     To generate only powers of 2, 5,6,7 10 can be written as poly_degrees=[2, (5,7), 10].
#     New in version 2.0: Modified to handle floats in addition to integers and ranges.

    result = set()
    if isinstance(value, (int, float)):
        result.add(value)  # If it's an integer or float, add it to the set
    elif isinstance(value, (tuple, list)) and len(value) == 2:
        start, end = value
        if all(isinstance(v, (int, float)) for v in (start, end)):
            # If both elements are int or float, generate the range.
            # Using a step of 1 for int range, approximate steps for float range.
            if isinstance(start, int) and isinstance(end, int):
                result.update(range(start, end + 1))
            else:
                # Generate numbers from start to end, with an increment that approximates integer steps.
                increment = 1 if start < end else -1
                while start <= end:
                    result.add(start)
                    start += increment
        else:
            raise ValueError(f"Both elements in the tuple/list must be integers or floats. Received: {value}")
    else:
        raise ValueError(f"Invalid input, expected int, float, or tuple/list with two integers or floats. Received: {value}")
    return result

In [ ]:
def create_candidate_func_w_func_names(DOF, poly_degrees=None, AFM_LJ_degrees=None, DMT_rep_degrees=None, DMT_att_degrees=None, n_frequencies=None, AFM_amp=None, a0_val = None, DMT_data = None,
                                       poly=True, poly_frac=False, sin=False, cos=False, exp_damp = False, z_b_val=None, exp_damp_degrees = None,
                                       sin_denominator=False, cos_denominator=False, AFM_LJ_pos_degree=False, AFM_LJ_pos_degree_degrees = None, AFM_LJ=False,
                                       AFM_LJ_damp=False, AFM_LJ_damp_degrees_e1 = None, AFM_LJ_damp_degrees_e2 = None,
                                       one_over_eta =False, one_over_eta_degrees = None, 
                                       DMT_rep = False, DMT_att = False, DMT_rep_smooth=False, DMT_smooth_viscoel_damp = False, 
                                       DMT_viscoel_damp=False, DMT_visc_elast_degrees_e1=None, DMT_visc_elast_degrees_e2=None):
    
    #Version 2.0: This function uses candidate functions created with lambda functions to generate a library of candidate functions for SINDy.

    #Parameters: DOF: Number of generalized coordinates in the system. (DOF=3 if we have x,y,phase)
    #            poly: Boolean to determine if normal polynomial terms should be included in the library. 
    #            poly_degrees: A list that states the degrees that the library should create for normal polynomial functions. Example: poly_degrees = [2, (4,6), 8] (generates 2nd, 4th,5th, 6th and 8th degree polynomials)
    #            special: Boolean to determine if a special AFM function should be created. Special AFM are of the form "(1-x)^(-degree)"
    #            special_degrees: A list that states the degrees that the library should create for special AFM polynomial functions. Example: special_degrees = [2, (4,6), 8] (generates 2nd, 4th,5th, 6th and 8th degree special polynomials)
    #            sin: Boolean to determine if a np.sin() should be created 
    #            cos: Boolean to determine if a np.cos() should be created
    #            sin_denominator: Boolean to determine if a "np.sin()^(-degree)"" should be created 
    #            cos_denominator: Boolean to determine if a "np.cos()^(-degree)"" should be created 
    #            n_frequencies: A list that states the frequencies that the library should create for sine, cosine functions. Example: n_frequencies = [2, (4,6), 8] (sin(2*x), sin(4*x), sin(5*x), sin(6*x), sin(8*x))
    #            AFM_amp: Is the y_bar value of an AFM equation. It is an integer value (a number).
    #            poly_frac: Boolean to determine if a rational polynomial "x^(-degree)" should be created.
    #            AFM_z: Boolean to fetermine if a AFM term with the shape of "1-x-y_bar*np.sin(t)" is created. 


    library_functions = []
    library_function_names = []

    if poly:
        degrees_to_generate = set()
        for degree_spec in poly_degrees:
            degrees_to_generate.update(to_ranges(degree_spec))

        for degree in sorted(degrees_to_generate):
            poly_func = make_poly(degree)
            poly_name = make_poly_name(degree)
            library_functions.append(poly_func)
            library_function_names.append(poly_name)

    if poly_frac:
        for degree in sorted(degrees_to_generate):
            poly_frac_func = make_poly_frac(degree)
            poly_frac_name = make_poly_frac_name(degree)
            library_functions.append(poly_frac_func)
            library_function_names.append(poly_frac_name)

    frequencies_to_generate = set()
    for frequency_spec in n_frequencies:
        frequencies_to_generate.update(to_ranges(frequency_spec))

    if sin:
        for frequency in sorted(frequencies_to_generate):
            func = make_sine(frequency)
            func_name = make_sine_name(frequency)
            library_functions.append(func)
            library_function_names.append(func_name)
            
            if sin_denominator:
                for degree in sorted(degrees_to_generate):
                    func_denom = make_denom_sine(frequency, degree)
                    func_name_denom = make_denom_sine_name(frequency, degree)
                    library_functions.append(func_denom)
                    library_function_names.append(func_name_denom)

    if cos:
        for frequency in sorted(frequencies_to_generate):
            func = make_cosine(frequency)
            func_name = make_cosine_name(frequency)
            library_functions.append(func)
            library_function_names.append(func_name)
            
            if cos_denominator:
                for degree in sorted(degrees_to_generate):
                    func_denom = make_denom_cosine(frequency, degree)
                    func_name_denom = make_denom_cosine_name(frequency, degree)
                    library_functions.append(func_denom)
                    library_function_names.append(func_name_denom)
    if AFM_LJ:
        degrees_to_generate = set()
        for degree_spec in AFM_LJ_degrees:
            degrees_to_generate.update(to_ranges(degree_spec))

        for degree in sorted(degrees_to_generate):
            func = make_AFM_LJ_func(degree)
            func_name = make_AFM_LJ_func_name(degree)
            library_functions.append(func)
            library_function_names.append(func_name)

    if AFM_LJ_pos_degree:
        degrees_to_generate = set()
        for degree_spec in AFM_LJ_pos_degree_degrees:
            degrees_to_generate.update(to_ranges(degree_spec))

        for degree in sorted(degrees_to_generate):
            func = make_AFM_LJ_pos_degree_func(degree=degree)
            func_name = make_AFM_LJ_pos_degree_name(degree=degree)
            library_functions.append(func)
            library_function_names.append(func_name)

    if AFM_LJ_damp:
        degrees_to_generate = set()
        for degree_spec in AFM_LJ_damp_degrees_e1:
            degrees_to_generate.update(to_ranges(degree_spec))
            
        degrees_to_generate_for_y = set()
        for degree_spec_for_y in AFM_LJ_damp_degrees_e2:
            degrees_to_generate_for_y.update(to_ranges(degree_spec_for_y))

        for degree in sorted(degrees_to_generate):
            for degree_2 in sorted(degrees_to_generate_for_y):
                DMT_visco_damp = make_AFM_LJ_damp_func(degree=degree, degree_2=degree_2)
                DMT_visco_damp_name = make_AFM_LJ_damp_name(degree=degree, degree_2=degree_2)
                library_functions.append(DMT_visco_damp)
                library_function_names.append(DMT_visco_damp_name)

    if DMT_att:
        degrees_to_generate = set()
        for degree_spec in DMT_att_degrees:
            degrees_to_generate.update(to_ranges(degree_spec))

        for degree in sorted(degrees_to_generate):
            func = make_AFM_DMT_att_func(x=DMT_data, a0=a0_val, degree=degree)
            func_name = make_AFM_DMT_att_name(a0=a0_val, degree=degree)
            library_functions.append(func)
            library_function_names.append(func_name)

    if DMT_rep:
        degrees_to_generate = set()
        for degree_spec in DMT_rep_degrees:
            degrees_to_generate.update(to_ranges(degree_spec))

        for degree in sorted(degrees_to_generate):
            func = make_AFM_DMT_rep_func(x=DMT_data, a0=a0_val, degree=degree)
            func_name = make_AFM_DMT_rep_name(a0=a0_val, degree=degree)
            library_functions.append(func)
            library_function_names.append(func_name)

    if one_over_eta:
        degrees_to_generate = set()
        for degree_spec in one_over_eta_degrees:
            degrees_to_generate.update(to_ranges(degree_spec))

        for degree in sorted(degrees_to_generate):
            func = make_1_over_eta_func(degree=degree)
            func_name = make_1_over_eta_func_name(degree=degree)
            library_functions.append(func)
            library_function_names.append(func_name)

    if DMT_viscoel_damp:
        degrees_to_generate = set()
        for degree_spec in DMT_visc_elast_degrees_e1:
            degrees_to_generate.update(to_ranges(degree_spec))
            
        degrees_to_generate_for_y = set()
        for degree_spec_for_y in DMT_visc_elast_degrees_e2:
            degrees_to_generate_for_y.update(to_ranges(degree_spec_for_y))

        for degree in sorted(degrees_to_generate):
            for degree_2 in sorted(degrees_to_generate_for_y):
                DMT_visco_damp = make_AFM_DMT_viscoel_damp_func(x=DMT_data, y=DMT_data, a0=a0_val, degree=degree, degree_2=degree_2)
                DMT_visco_damp_name = make_AFM_DMT_viscoel_damp_name(a0=a0_val, degree=degree, degree_2=degree_2)
                library_functions.append(DMT_visco_damp)
                library_function_names.append(DMT_visco_damp_name)

    if exp_damp:
        degrees_to_generate = set()
        for degree_spec in exp_damp_degrees:
            degrees_to_generate.update(to_ranges(degree_spec))

        for degree in sorted(degrees_to_generate):
            func = make_exp_damping_func(z_b=z_b_val, degree=degree)
            func_name = make_exp_damping_name(z_b=z_b_val, degree=degree)
            library_functions.append(func)
            library_function_names.append(func_name)

    return library_functions, library_function_names

In [ ]:
def get_index_of_cand_function(SINDy_model, cand_function_to_check):
    index = None
    for idx, cand_func in enumerate(SINDy_model.get_feature_names()):
        if cand_func == cand_function_to_check:
            index= idx

    return index

## Functions for Candidate Function Reduction Analysis

In [ ]:
def replace_feature_names(feature_string, name_mapping):
    # Use regular expressions to replace all occurrences of variable names
    for old_name, new_name in name_mapping.items():
        # Double the backslashes in new_name for use in re.sub
        safe_new_name = new_name.replace('\\', '\\\\')
        feature_string = re.sub(r'\b' + re.escape(old_name) + r'\b', safe_new_name, feature_string)
    return feature_string

## Functions to generate Clusters

In [ ]:
def define_cluster_centers(x_train_mult_trajectories, num_points_high_res, sub_sample_val, filter_vel_max_val, index_to_plot, centers_loc, multiple_init_cond = False, plot_file_name = None, plot=False, save_plot=False):

    # Version 1.0: This function takes an array of arrays with multiple initial conditions and places points along the phase space that later would be used as
    # cluster centers
    #
    # Parameters: x_train_mult_trajectories: List of arrays, where each array contain the time signal of an initial condition with shape [num_of_data_points, 3] 
    #             num_points_high_res: Integer defining the number of high-resolution points to be used when interpolating the trajectory in the phase space.
    #             sub_sample_val: Integer defining the step size for subsampling the high-resolution points to reduce the total number of cluster centers.
    #             filter_vel_max_val: Float defining the maximum velocity threshold, below which points are considered valid for being cluster centers.
    #             index_to_plot: Integer index specifying which trajectory from the list should be plotted (used when `plot` is True).
    #             centers_loc: Integer determining the location of centers; typically 2 for synthetic data and 5 or 6 for experimental data, affecting how the starting and ending points of closed orbits are calculated.
    #             multiple_init_cond: Boolean indicating if centers are being defined for multiple initial conditions at the same time. Default is False.
    #             plot_file_name: String specifying the filename for the plot, if saved. Relevant when `save_plot` is True.
    #             plot: Boolean to control whether to plot the phase space with the defined centers. Default is False.
    #             save_plot: Boolean to control whether to save the plot to a file. Default is False.

    cluster_centers_x = []
    cluster_centers_y = []

    if multiple_init_cond:
        for analyzed_trajectory in x_train_mult_trajectories:
            init_closed_orbit = (analyzed_trajectory.shape[0]//centers_loc) - 1000  #centers_loc is usually 2 for synthetic data
            end_closed_orbit = (analyzed_trajectory.shape[0]//centers_loc) + 1000   #centers_loc is usually 5 or 6 for experimental data

            #To extract the last closed orbit
            x = analyzed_trajectory[init_closed_orbit:end_closed_orbit,0] #[-650:,0]
            v = analyzed_trajectory[init_closed_orbit:end_closed_orbit,1]

            #Calculate the arc length for parameterization
            arc_lengths = np.zeros_like(x)
            arc_lengths[1:] = np.cumsum(np.sqrt(np.diff(x)**2 + np.diff(v)**2))

            #Parameterize the orbit by arc length
            spline_x = CubicSpline(arc_lengths, x)
            spline_v = CubicSpline(arc_lengths, v)

            # Initial high-resolution sampling
            high_res_lengths = np.linspace(arc_lengths[0], arc_lengths[-1], num_points_high_res)
            high_res_x = spline_x(high_res_lengths)
            high_res_v = spline_v(high_res_lengths)

            # Subsample every nth point, n determines the new spacing
            n = sub_sample_val 
            subsampled_x = high_res_x[::n]
            subsampled_v = high_res_v[::n]

            # Filter to keep only points with negative velocity
            negative_velocity_mask = subsampled_v < filter_vel_max_val
            filtered_x = subsampled_x[negative_velocity_mask]
            filtered_v = subsampled_v[negative_velocity_mask]
            
            cluster_centers_x.append(filtered_x)
            cluster_centers_y.append(filtered_v)

        if plot:
            fig, axs = plt.subplots(1, 1, figsize=(10, 8)) #[-650:,0]
            axs.plot(x_train_mult_trajectories[index_to_plot][:,0], x_train_mult_trajectories[index_to_plot][:,1], color='black', label='Last closed orbit', alpha = 0.7)
            axs.scatter(cluster_centers_x[index_to_plot], cluster_centers_y[index_to_plot], color='blue', label='Cluster Centers')
            axs.set_title('Phase Space Plot with Cluster Center \n Points Having Negative Velocity', fontsize=20)
            axs.set_xlabel(r'Displacement $\bar{\eta}_1$ [-]', fontsize=20)
            axs.set_ylabel(r' Velocity $\bar{\eta}_2$ [-]', fontsize=20)
            axs.legend(loc = 'upper left')
            axs.tick_params(axis='both', which='major', labelsize=20)  # Customize the font size of the tick labels
            axs.tick_params(axis='both', which='major', labelsize=20)  # Customize the font size of the tick label

            if save_plot:
                fig.savefig(f'{plot_file_name}.png', transparent=True, dpi=300, bbox_inches='tight')
            plt.show()
    
    else:
        analyzed_trajectory = x_train_mult_trajectories
        init_closed_orbit = (analyzed_trajectory.shape[0]//centers_loc) - 1000  #centers_loc is usually 2 for synthetic data
        end_closed_orbit = (analyzed_trajectory.shape[0]//centers_loc) + 1000   #centers_loc is usually 5 or 6 for experimental data

        x = analyzed_trajectory[init_closed_orbit:end_closed_orbit,0]
        v = analyzed_trajectory[init_closed_orbit:end_closed_orbit,1]  #[50000:53000]

        #Calculate the arc length for parameterization
        arc_lengths = np.zeros_like(x)
        arc_lengths[1:] = np.cumsum(np.sqrt(np.diff(x)**2 + np.diff(v)**2))

        #Parameterize the orbit by arc length
        spline_x = CubicSpline(arc_lengths, x)
        spline_v = CubicSpline(arc_lengths, v)

        # Initial high-resolution sampling
        high_res_lengths = np.linspace(arc_lengths[0], arc_lengths[-1], num_points_high_res)
        high_res_x = spline_x(high_res_lengths)
        high_res_v = spline_v(high_res_lengths)

        # Subsample every nth point, n determines the new spacing
        n = sub_sample_val 
        subsampled_x = high_res_x[::n]
        subsampled_v = high_res_v[::n]

        # Filter to keep only points with negative velocity
        negative_velocity_mask = subsampled_v < filter_vel_max_val
        filtered_x = subsampled_x[negative_velocity_mask]
        filtered_v = subsampled_v[negative_velocity_mask]
        
        cluster_centers_x.append(filtered_x)
        cluster_centers_y.append(filtered_v)

        if plot:
            fig, axs = plt.subplots(1, 1, figsize=(10, 8))
            axs.plot(x_train_mult_trajectories[:,0], x_train_mult_trajectories[:,1], color='black', label='HOPG Expeimental Data', linewidth=0.5, alpha=0.7)
            axs.scatter(cluster_centers_x[0], cluster_centers_y[0], color='blue', label='Cluster Centers')
            axs.set_title('Phase Space Plot with Cluster Center \n Points Having Negative Velocity', fontsize=20)
            axs.set_xlabel(r'Displacement $\bar{\eta}_1$ [-]', fontsize=20)
            axs.set_ylabel(r' Velocity $\bar{\eta}_2$ [-]', fontsize=20)
            axs.legend(loc = 'upper left')
            axs.tick_params(axis='both', which='major', labelsize=20)  # Customize the font size of the tick labels
            axs.tick_params(axis='both', which='major', labelsize=20)  # Customize the font size of the tick label

            if save_plot:
                fig.savefig(f'{plot_file_name}.png', transparent=True, dpi=300, bbox_inches='tight')
            plt.show()

    return cluster_centers_x, cluster_centers_y

In [ ]:
def generate_fixed_clusters_from_centers(cluster_size, x_train_mult_trajectories, KNN_neighbors_num, mult_traj_cluster_centers_x, mult_traj_cluster_centers_y, KNN_radius = 1.0, multiple_init_cond=False):

    # Version 1.0: This function works by fitting a nearest neighbors classifier to the trajectory points and using it to find the closest neighbors around predefined
    #              cluster centers specified by the mult_traj_cluster_centers_x and mult_traj_cluster_centers_y parameters. Clusters are defined by including a fixed 
    #              number of points around each center, dictated by the cluster_size parameter. If multiple_init_cond is set to True, the function processes multiple trajectories independently.
    # 
    # Parameters:  cluster_size: Integer specifying the number of data points to include in each cluster around the cluster center.
    #              x_train_mult_trajectories: List of arrays, where each array is a trajectory with shape [num_of_data_points, dimensions] typically dimensions being 3 for 3D data.
    #              KNN_neighbors_num: Integer specifying the number of nearest neighbors to consider for determining the cluster membership.
    #              mult_traj_cluster_centers_x: List of lists containing the x-coordinates of cluster centers for multiple trajectories.
    #              mult_traj_cluster_centers_y: List of lists containing the y-coordinates of cluster centers for multiple trajectories.
    #              KNN_radius: Float specifying the radius within which to search for nearest neighbors. Default is 1.0.
    #              multiple_init_cond: Boolean indicating if the function should handle multiple trajectories simultaneously. Default is False.

    #Returns:
    #    mult_traj_clusters_sections: List of lists, where each sublist contains arrays of sections of trajectories that are part of a cluster.
    #    mult_traj_clusters_dots_list: List of lists, where each sublist contains the indices of the points in the trajectory that form a cluster around a center.

    mult_traj_clusters_dots_list = []
    mult_traj_clusters_sections = []

    if multiple_init_cond:
        for i, analyzed_trajectory in enumerate(x_train_mult_trajectories):
            KNN_traj_for_fit_temp = np.vstack((analyzed_trajectory[:,0], analyzed_trajectory[:,1])).T
            neigh = NearestNeighbors(n_neighbors=KNN_neighbors_num, radius = KNN_radius)
            neigh.fit(KNN_traj_for_fit_temp)

            single_traj_cluster_dots_list = []
            for index_center in range(len(mult_traj_cluster_centers_x[i])):
                cluster_dots_temp = neigh.kneighbors([[mult_traj_cluster_centers_x[i][index_center], mult_traj_cluster_centers_y[i][index_center]]], return_distance=False)
                single_traj_cluster_dots_list.append(cluster_dots_temp[0]) #this is always index [0] because neigh.kneighbors() gives a (1,100) array by default. So this gives (100,) already
            mult_traj_clusters_dots_list.append(single_traj_cluster_dots_list)
            
        for j, single_traj_clusters_dots in enumerate(mult_traj_clusters_dots_list):
            analyzed_trajectory = x_train_mult_trajectories[j]

            single_traj_clusters_sections = []
            for single_cluster_dots in single_traj_clusters_dots:
                single_traj_single_cluster_sections = []
                for single_cluster_dot in single_cluster_dots:
                    single_trajectory_section = np.vstack((analyzed_trajectory[(single_cluster_dot-cluster_size):(single_cluster_dot+cluster_size),0], analyzed_trajectory[(single_cluster_dot-cluster_size):(single_cluster_dot+cluster_size),1], analyzed_trajectory[(single_cluster_dot-cluster_size):(single_cluster_dot+cluster_size),2])).T
                    single_traj_single_cluster_sections.append(single_trajectory_section)
                single_traj_clusters_sections.append(single_traj_single_cluster_sections)
            mult_traj_clusters_sections.append(single_traj_clusters_sections) 

    else:
        analyzed_trajectory = x_train_mult_trajectories
        KNN_traj_for_fit_temp = np.vstack((analyzed_trajectory[:,0], analyzed_trajectory[:,1])).T
        neigh = NearestNeighbors(n_neighbors=KNN_neighbors_num, radius=KNN_radius)
        neigh.fit(KNN_traj_for_fit_temp)

        single_traj_cluster_dots_list = []
        for index_center in range(len(mult_traj_cluster_centers_x)):
            cluster_dots_temp = neigh.kneighbors([[mult_traj_cluster_centers_x[index_center], mult_traj_cluster_centers_y[index_center]]], return_distance=False)
            single_traj_cluster_dots_list.append(cluster_dots_temp[0]) #this is always index [0] because neigh.kneighbors() gives a (1,100) array by default. So this gives (100,) already
        mult_traj_clusters_dots_list.append(single_traj_cluster_dots_list)

        single_traj_clusters_dots = mult_traj_clusters_dots_list[0]
        single_traj_clusters_sections = []
        for single_cluster_dots in single_traj_clusters_dots:
            single_traj_single_cluster_sections = []
            for single_cluster_dot in single_cluster_dots:
                single_trajectory_section = np.vstack((analyzed_trajectory[(single_cluster_dot-cluster_size):(single_cluster_dot+cluster_size),0], analyzed_trajectory[(single_cluster_dot-cluster_size):(single_cluster_dot+cluster_size),1], analyzed_trajectory[(single_cluster_dot-cluster_size):(single_cluster_dot+cluster_size),2])).T
                single_traj_single_cluster_sections.append(single_trajectory_section)
            single_traj_clusters_sections.append(single_traj_single_cluster_sections)
        mult_traj_clusters_sections.append(single_traj_clusters_sections) 

    return mult_traj_clusters_sections, mult_traj_clusters_dots_list

In [ ]:
def filter_close_points(x_points, y_points, threshold=0.2):
    filtered_x = []
    filtered_y = []

    for x, y in zip(x_points, y_points):
        # Calculate distances from the current point to all points in the filtered lists
        if filtered_x: 
            distances = np.sqrt((np.array(filtered_x) - x) ** 2 + (np.array(filtered_y) - y) ** 2)
            if np.all(distances > threshold):
                filtered_x.append(x)
                filtered_y.append(y)
        else:
            filtered_x.append(x)
            filtered_y.append(y)

    return np.array(filtered_x), np.array(filtered_y)

In [ ]:
def get_cluster_centroids_from_data(analyzed_trajectories, neighbors_num, normalize_data=False):

    x = analyzed_trajectories[:, 0]        # All displacement values
    y = analyzed_trajectories[:, 1]        # All velocity values
    times = analyzed_trajectories[:, 2]    # All time values

    if normalize_data:
        # Normalize the data
        x_data = x / max(x)
        y_data = y / max(y)
    else:
        x_data = x 
        y_data = y

    # Combine into a single array of coordinates
    coordinates = np.vstack((x_data, y_data)).T

    # Initialize NearestNeighbors with the number of neighbors 'neighbors_num'
    nn = NearestNeighbors(n_neighbors=neighbors_num, algorithm='auto').fit(coordinates)
    # Find the k-neighbors of each point
    distances, indices = nn.kneighbors(coordinates)

    # Calculate cluster_centroid
    cluster_centroids = []
    for idx in indices:
        cluster_points = coordinates[idx]
        centroid = cluster_points.mean(axis=0)
        cluster_centroids.append(centroid)

    cluster_centroids = np.array(cluster_centroids)

    return cluster_centroids

## Functions for training SINDy models

In [ ]:
def generate_serial_number(index, cluster, lambda_value, nu_value):
   # Version 1.0: This function generates a unique serial number for when a trained SINDy model is saved.

   # Parameters:  index: Integer specifying the number of data points to include in each cluster around the cluster center.
   #              cluster: number of the cluster being analyzed
   #              lambda_value: lambda value used for the ContrainedSR3() optimizer
   #              nu_value: nu value used for the ContrainedSR3() optimizer

    date_str = datetime.now().strftime("%d-%m-%y")  # Current date in dd-mm-yy format
    unique_number = f"{index:03d}"  # Zero-padded 3-digit number
    return f"{date_str}-{unique_number}-{cluster}_cluster_{cluster}_lambda_{lambda_value}_nu_{nu_value}_"

In [ ]:
def fit_constrained_SINDy_model(candidate_func_library, model_feature_names, constraint_rhs_array, constraint_lhs_array, trajectories_data, lambda_val, traject_dt, nu_val, ensemble = False, multiple_trajectories = False, model_filename = None, save_model = False):
    # Version 1: This function is used to fit/train a SINDy model using ConstrainedSINDy with ConstrainedSR3. 
    #
    # Parameters: candidate_func_library: Number of lambda values to generate
    #             model_feature_names: smallest order of magnitude for generated lambdas
    #             constraint_rhs_array: numpy array containing the right hand side constraints for ContrainedSR3()
    #             constraint_lhs_array: numpy array containing the left hand side constraints for ContrainedSR3()
    #             trajectories_data: The time-series data from the system to be analyzed. 
    #             lambda_val: lambda value used for the ContrainedSR3() optimizer
    #             nu_val: nu value used for the ContrainedSR3() optimizer
    #             traject_dt: time step of the time-series data.
    #             model_filename: Name for the .dill file used to save the trained model if needed.

    model_optimizer = ps.ConstrainedSR3(constraint_rhs=constraint_rhs_array, constraint_lhs=constraint_lhs_array, threshold=lambda_val, nu = nu_val, tol=1e-9, thresholder="l0", max_iter=10000)
    constrained_model = ps.SINDy(optimizer=model_optimizer, feature_library=candidate_func_library, feature_names=model_feature_names)
    constrained_model.fit(trajectories_data, t=traject_dt, ensemble=ensemble, multiple_trajectories = multiple_trajectories, quiet=True)

    if save_model:
        with open(model_filename + '.dill', 'wb') as f: #saves the model with the .dill extension within the str name, for clarity on how to open the models later
            dill.dump(constrained_model, f)
    
    return constrained_model

In [ ]:
def generate_random_lambda_n_nu(num_values, smaller_order_magnitude, bigger_order_magnitude, randomize=False):
    # Version 1: This function generates an array of lambda coeffieints to be used as threshold values for a 
    #            SINDy algorithm. I genereates lambda values from a small order to a big order of magnitude
    #            by creating logaritmically spaces values. 
    # Parameters: num_values: Number of lambda values to generate
    #             smaller_order_magnitude: smallest order of magnitude for generated lambdas
    #             bigger_order_magnitude: biggest order of magnitude for generated lambdas
    #             randomize: Boolean than determines if the order of lambdas is randomized in the final list.

    # Generate logarithmically spaced values
    log_values = np.linspace(smaller_order_magnitude, bigger_order_magnitude, num_values)
    values = 10**log_values

    if randomize:
        np.random.shuffle(values)
    return values

## Functions for opening trained models

In [ ]:
def import_trained_models_from_folder(folder_path, folder_name):
    # Version 1: This function imports all saved SINDy models that are within a folder 
    # Parameters: folder_path: Path where the folder is located
    #             folder_name: Name of the folder with the models to upload

    # Define the path to the folder
    path = f'{folder_path}/{folder_name}/*.dill'
    files = glob.glob(path)

    # Initialize lists and dictionary
    files_list = []  # List of file names
    loaded_models_list = []  # List of loaded models
    models_dict = {}  # Dictionary of models with file names as keys

    # Iterate through the files
    for file in files:
        # Extract the file name
        file_name = file.replace(f'{folder_path}/{folder_name}/', "")
        files_list.append(file_name)

        # Load the model
        with open(file, 'rb') as f:
            loaded_model = dill.load(f)
        loaded_models_list.append(loaded_model)

        # Add to the dictionary
        models_dict[file_name] = loaded_model

    # Return all three structures
    return files_list, loaded_models_list, models_dict

In [ ]:
def import_single_trained_model_from_folder(folder_path, folder_name, file_name):
#   Version 1: This function imports a single SINDy trained model that has been saved in a .dill format
#
#   Parameters: folder_name: path with the name of the folder where the model to be uploaded is located
#               file_name: the name of the model needed to be uploaded
    # Define the full path to the file
    path = f'{folder_path}/{folder_name}/{file_name}'

    # Check if the file exists
    if not os.path.exists(path):
        raise FileNotFoundError(f"No file found at {path}")

    # Load the model
    with open(path, 'rb') as f:
        loaded_model = dill.load(f)
    
    return loaded_model

In [ ]:
def import_trained_models_from_folder_to_dataframe(folder_path, folder_name, new_dataframe, number_of_parsimony_eq):
#    Version 1: This function imports all saved SINDy models that are within a folder and places them in a Pandas dataframe
#    Parameters: folder_path: Path where the folder is located
#                folder_name: Name of the folder with the models to upload

    # Define the path to the folder
    path = f'{folder_path}/{folder_name}/*.dill'
    files = glob.glob(path)

    # Initialize lists and dictionary
    files_list = []  # List of file names
    loaded_models_list = []  # List of loaded models
    models_dict = {}  # Dictionary of models with file names as keys

    # Iterate through the files
    for file in files:
        # Extract the file name
        file_name = file.replace(f'{folder_path}/{folder_name}/', "")
        files_list.append(file_name)

        # Load the model
        with open(file, 'rb') as f:
            loaded_model = dill.load(f)
        loaded_models_list.append(loaded_model)

        headings_from_file = file_name.split('_')

        #Checking parsimony:
        if number_of_parsimony_eq == 1: 
            eq_fund_in_table = convert_to_table(loaded_model.equations(precision=9)[0])
            fund_eq_len = eq_fund_in_table.count()

        if number_of_parsimony_eq == 2: 
            eq_fund_in_table = convert_to_table(loaded_model.equations(precision=9)[1])
            fund_eq_len = eq_fund_in_table.count()

        # Add to the dictionary
        models_dict[file_name] = loaded_model
        new_row = {new_dataframe.columns[0]:[loaded_model],
                    new_dataframe.columns[1]:[headings_from_file[2].capitalize()],
                    new_dataframe.columns[2]:[headings_from_file[4]],
                    new_dataframe.columns[3]:[headings_from_file[6]],
                    new_dataframe.columns[4]:[headings_from_file[8]],
                    new_dataframe.columns[5]:[headings_from_file[10]],
                    new_dataframe.columns[6]:[fund_eq_len]} 
        new_dataframe = pd.concat([new_dataframe, pd.DataFrame(new_row)], ignore_index=True)

    return new_dataframe

In [ ]:
def mark_unique_models_in_dataframe(df):
#     Version 1: This function marks models in the DataFrame as unique or not based on their coefficients and active terms.
#     Parameters: df (pd.DataFrame): DataFrame containing trained models.
#                 folder_name: Name of the folder with the models to upload
#.    Returns: pd.DataFrame: Updated DataFrame with an additional 'Is Unique' column.

    encountered_models = set()
    uniqueness_flags = []

    for idx, row in df.iterrows():
        model = row['Candidate Model']  # Extract the model

        # Extract coefficients and candidate functions
        coefficients = np.round(model.coefficients(), decimals=7)  # Normalize coefficients
        terms = model.get_feature_names()  # Full library

        # Extract only active terms
        active_terms = [
            terms[i] for i, coef in enumerate(coefficients) if not np.allclose(coef, 0)
        ]
        active_coefficients = [
            tuple(coef) for coef in coefficients if not np.allclose(coef, 0)
        ]

        # Create a unique signature for the model
        model_signature = (tuple(active_coefficients), tuple(active_terms))

        # Check for uniqueness
        if model_signature not in encountered_models:
            encountered_models.add(model_signature)
            uniqueness_flags.append(True)  # Mark as unique
        else:
            uniqueness_flags.append(False)  # Mark as not unique

    # Add the uniqueness column to the DataFrame
    df['Is Unique'] = uniqueness_flags
    return df

In [ ]:
def check_unique_trained_models_from_dataframe(dataframe_with_models):

    unique_models_dict = {}  
    unique_models_list = []
    unique_model_names = []
    encountered_models = set()

    for index, loaded_model in enumerate(dataframe_with_models['Candidate Model']):

        # Extract coefficients and candidate functions
        coefficients = np.round(loaded_model.coefficients(), decimals=7)  # Normalize coefficients
        terms = loaded_model.get_feature_names()  # Full library

        # Extract only active terms
        active_terms = [
            terms[i] for i, coef in enumerate(coefficients) if not np.allclose(coef, 0)
        ]
        active_coefficients = [
            tuple(coef) for coef in coefficients if not np.allclose(coef, 0)
        ]

        # Create a unique signature for the model
        model_signature = (tuple(active_coefficients), tuple(active_terms))

        model_serial_number = dataframe_with_models.iloc[index]['Serial No.']

        # Check for uniqueness
        if model_signature not in encountered_models:
            encountered_models.add(model_signature)
            unique_models_dict[model_serial_number] = loaded_model  
            unique_models_list.append(loaded_model)
            unique_model_names.append(model_serial_number)

    return unique_model_names, unique_models_list, unique_models_dict

## Functions for Simulating models

In [ ]:
def simulate_F_ts_from_found_model_DMT(found_model, simulated_model):
#   Version 1: This function receives a SINDy model that has been fit before. 
#
#   Parameters: found_model: This is a SINDy that has been previously trained. 
#               simulated_model: The found SINDy model that has been simulated with model.simulate()
    
    model_copy = copy.deepcopy(found_model) # Create a deep copy of the original model to avoid modifying it

    coefficients = model_copy.coefficients()
    library_terms = model_copy.get_feature_names()

    # Modify the coefficients of the copied model
    for i, term in enumerate(library_terms):
        if term == 'e1':  # Removes stiffness terms out of the e2' equation only. 
            coefficients[1, i] = 0 
        elif term == 'e2': # Removes damping terms out of the e2' equation only. 
            coefficients[1, i] = 0
        elif term == 'sin(1*phase)': # Removes Forcing terms out of the e2' equation only. 
            coefficients[1, i] = 0
        elif term == '1': # Removes constant terms out of the e2' equation only. 
            coefficients[1, i] = 0

    model_copy.coefficients_ = coefficients

    force_list = []
    simulated_F_ts = []

    # Simulate using the modified copy
    for current_point in simulated_model:
        model_sim_eval_temp = model_copy.predict(current_point[np.newaxis, :])[0]
        force_list.append(model_sim_eval_temp[0])
        simulated_F_ts.append(-model_sim_eval_temp[1])

    return simulated_F_ts

In [ ]:
def calculate_true_F_ts_DMT_found(sim_disp_data, a0):
    #Version 1.0: This function takes the array of simulated data and calculates the Tip-Sample Force using the DMT model where the found equations were written manually
    #             from the found equations by AFM-SINDy

    #Parameters: - sim_disp_data: Synthetic data from a found DMT model.
    #              It can be either a single measurement in an np.array or multiple trajectories in a list.
    #            - a0: found intermolecular distance.
    if isinstance(sim_disp_data, np.ndarray):
        F_ts_true_array = np.zeros_like(sim_disp_data[:,0],dtype=float)
        for i in range(len(sim_disp_data)):
            x_elem = sim_disp_data[i, 0]
            y_elem = sim_disp_data[i, 1]
            F_ts_temp = calculate_AFM_DMT_Force_found(eta_1=x_elem, eta_2 = y_elem, a0=a0)
            F_ts_true_array[i] = F_ts_temp
        return -1*F_ts_true_array 

    if isinstance(sim_disp_data, list):
        F_ts_true_list = []
        F_ts_true_array = np.zeros_like(sim_disp_data[0][:,0],dtype=float)
        for trajectory in sim_disp_data:
            for i in range(len(trajectory)):
                x_elem = trajectory[i, 0]
                y_elem = trajectory[i, 1]
                F_ts_temp = calculate_AFM_DMT_Force_found(eta_1=x_elem, eta_2 = y_elem, a0=a0)
                F_ts_true_array[i] = F_ts_temp
            F_ts_true_list.append(F_ts_true_array) 

        return -1*F_ts_true_list

In [ ]:
def simulate_results_from_cluster(model_to_simulate, validation_trajectory_section, full_validation_trajectory, solve_ivp_method, dt, t_steps_beyond_cluster, a0, model_type, C1=None, C2=None, C3=None, disp_plot_file_name= None, vel_plot_file_name = None, force_plot_file_name= None, plot = False, save=False):
#   Version 1: This function takes a model that has been trained and simulates it in time from the cluster from which it was found. 
# 
#   Parameters: model_to_simulate: Is the fit SINDy found model to simulate.
#               validation_trajectory_section: Is the array single trajectory section within "full_validation_trajectory" that we can to use as validation for the simulated SINDy found model.
#                                              Typically is a section of a cluster. Therefore, it must have a shape of (num_of_data, 3). (dist, vel, phase). Example: mult_traj_clusters_sections[traject][cluster_att][10]
#               full_validation_trajectory: Is the complete trajectory starting from transient to steady state that can be used as validation. It is used to simulate a bit furhter than the cluster size. Example: x_train_DMT_mult_traj[traject]
#               solve_ivp_method: Is a string to choose the method to perform the numerical integration like 'RK45' or 'BDF'.
#               disp_plot_file_name: Is the name for the displacement figure if Save=True.
#               force_plot_file_name: Is the name for the displacement figure if Save=True.

    t_simulate = np.arange(validation_trajectory_section[0][2], validation_trajectory_section[0][2]+(t_steps_beyond_cluster*dt), dt) #This is the reason why it only works for attractive clusters so far. t_simulate can not go further than att region "+2.5"
    init_cond = validation_trajectory_section[0]

    simulated_model = model_to_simulate.simulate(init_cond, t_simulate, integrator_kws={"method": solve_ivp_method, "rtol": 1e-5, "atol": 1e-7})  #"rtol": 1e-5, "atol": 1e-7})

    init_cond_index = np.where(full_validation_trajectory[:,2] == t_simulate[0])[0][0] #Find where in the full trajectory the t_simulate starts for plotting and giving a score
    next_index = init_cond_index + simulated_model.shape[0] #Find where in the full trajectory the t_simulate ends or plotting and giving a score

    extended_validation_traject = np.vstack((full_validation_trajectory[init_cond_index:next_index,0], full_validation_trajectory[init_cond_index:next_index,1], full_validation_trajectory[init_cond_index:next_index,2])).T

    if model_type == 'Normal DMT':
        F_ts_true = calculate_true_F_ts_DMT(sim_disp_data=full_validation_trajectory[init_cond_index:next_index], a0=a0, C1=C1, C2=C2)
        F_ts_sim = simulate_F_ts_from_found_model_DMT(model_to_simulate, simulated_model)
    elif model_type == 'Viscoelastic DMT':
        F_ts_true = calculate_true_F_ts_DMT_viscoelast_damp(sim_disp_data=full_validation_trajectory[init_cond_index:next_index], a0=a0, C1=C1, C2=C2, C3=C3)
        F_ts_sim = simulate_F_ts_from_found_model_DMT(model_to_simulate, simulated_model)

    if plot: 
        #Plots displacement:
        fig, axs = plt.subplots(1, 1, figsize=(10, 8))
        axs.plot(extended_validation_traject[:,2], extended_validation_traject[:,0], color = 'green', label = r'True $\eta_1$')
        axs.plot(t_simulate, simulated_model[:,0], '--', color = 'black', label = r'Found $\eta_1$')
        axs.plot(validation_trajectory_section[:,2], validation_trajectory_section[:,0], '-', color = 'red', linewidth = 3, alpha = 0.5, label = r'Cluster Extension')
        axs.set_title('Comparison between true and found dynamics', fontsize=25)
        axs.set_xlabel(r'Time [-]', fontsize=25)
        axs.set_ylabel(r'Displacement [-]', fontsize=25)
        axs.legend(loc='best', fontsize=15)
        axs.set_xlim([t_simulate[0], t_simulate[-1]])
        axs.tick_params(axis='both', which='major', labelsize=25)  
        axs.tick_params(axis='both', which='major', labelsize=25)  
        if save:
            plt.savefig(disp_plot_file_name + '.png', dpi=300, bbox_inches='tight', transparent=True)

        #Plots Velocity:
        fig, axs = plt.subplots(1, 1, figsize=(10, 8))
        axs.plot(extended_validation_traject[:,2], extended_validation_traject[:,1], color = 'green', label = r'True $\eta_2$')
        axs.plot(t_simulate, simulated_model[:,1], '--', color = 'black', label = r'Found $\eta_2$')
        axs.plot(validation_trajectory_section[:,2], validation_trajectory_section[:,1], '-', color = 'red', linewidth = 3, alpha = 0.5, label = r'Cluster Extension')
        axs.set_title('Comparison between true and found dynamics', fontsize=25)
        axs.set_xlabel(r'Time [-]', fontsize=25)
        axs.set_ylabel(r'Velocity [-]', fontsize=25)
        axs.legend(loc='best', fontsize=15)
        axs.set_xlim([t_simulate[0], t_simulate[-1]])
        axs.tick_params(axis='both', which='major', labelsize=25)  
        axs.tick_params(axis='both', which='major', labelsize=25)  
        if save:
            plt.savefig(vel_plot_file_name + '.png', dpi=300, bbox_inches='tight', transparent=True)

        #Checking parsimony:
        eq_fund_tab_att_pareto = convert_to_table(model_to_simulate.equations(precision=9)[1])
        eq_len_att_pareto = eq_fund_tab_att_pareto.count()

        print('')
        print('Equation with length: ' + str(eq_len_att_pareto))
        model_to_simulate.print(precision=9)
        print('')

        #plot the force calculations:
        fig = plt.figure()
        plt.plot(t_simulate, F_ts_true, color = 'green', label = r'True $F_{ts}$')
        plt.plot(t_simulate, F_ts_sim, '--', color = 'black', label = r'Found $F_{ts}$')
        plt.title('Comparison between true force and found force')
        plt.xlabel('Time [-]')
        plt.ylabel('Tip-Sample Force [-]')
        plt.legend(loc='best')
        if save:
            plt.savefig(force_plot_file_name + '.png', dpi=300, bbox_inches='tight', transparent=True)
    
    return simulated_model, extended_validation_traject, F_ts_true, F_ts_sim

In [ ]:
def simulate_results_from_cluster_until_ts(model_to_simulate, validation_trajectory_section, full_validation_trajectory, solve_ivp_method, dt, t_steps_beyond_cluster, threshold_for_ts, a0, model_type, C1=None, C2=None, C3=None, F_xlim = None, print_switch_index_val = True, disp_plot_file_name= None, vel_plot_file_name = None, force_plot_file_name= None, plot = False, save=False):
#   Version 1: This function takes a model that has been trained and simulates it in time from the cluster from which it was found until a divergence time ts.
# 
#   Parameters: model_to_simulate: Is the fit SINDy found model to simulate.
#               validation_trajectory_section: Is the array single trajectory section within "full_validation_trajectory" that we can to use as validation for the simulated SINDy found model.
#                                              Typically is a section of a cluster. Therefore, it must have a shape of (num_of_data, 3). (dist, vel, phase). Example: mult_traj_clusters_sections[traject][cluster_att][10]
#               full_validation_trajectory: Is the complete trajectory starting from transient to steady state that can be used as validation. It is used to simulate a bit furhter than the cluster size. Example: x_train_DMT_mult_traj[traject]
#               solve_ivp_method: Is a string to choose the method to perform the numerical integration like 'RK45' or 'BDF'.
#               disp_plot_file_name: Is the name for the displacement figure if Save=True.
#               force_plot_file_name: Is the name for the displacement figure if Save=True.

    t_simulate = np.arange(validation_trajectory_section[0][2], validation_trajectory_section[0][2]+(t_steps_beyond_cluster*dt), dt) #This is the reason why it only works for attractive clusters so far. t_simulate can not go further than att region "+2.5"
    init_cond = validation_trajectory_section[0]

    simulated_model = model_to_simulate.simulate(init_cond, t_simulate, integrator_kws={"method": solve_ivp_method, "rtol": 1e-5, "atol": 1e-7})

    init_cond_index = np.where(full_validation_trajectory[:,2] == t_simulate[0])[0][0] #Find where in the full trajectory the t_simulate starts for plotting and giving a score
    next_index = init_cond_index + simulated_model.shape[0] #Find where in the full trajectory the t_simulate ends or plotting and giving a score

    extended_validation_traject = np.vstack((full_validation_trajectory[init_cond_index:next_index,0], full_validation_trajectory[init_cond_index:next_index,1], full_validation_trajectory[init_cond_index:next_index,2])).T

    # Extract time vector from validation_data (assuming it's the third column)
    tvec = extended_validation_traject[:, 2]
    t_simulate = simulated_model[:,2]

    # Ensure the lengths match
    if extended_validation_traject.shape != simulated_model.shape:
        raise ValueError('Validation data and model predictions must have the same shape.' + str(extended_validation_traject.shape) + ' and ' + str(simulated_model.shape))

    tlength, nterms = simulated_model.shape  # Time length and number of variables
    abserror = np.abs(extended_validation_traject - simulated_model)  # Element-wise absolute error

    # Initialize outputs
    tdiv = np.zeros(nterms-1) 
    abserror_avg = np.zeros(nterms-1) #intentionally excluting time, as I do not need discrepancies in time
    rmse = np.zeros(nterms-1)
    abserror_over_time = np.zeros(nterms-1)

    # Loop over each state variable
    for state_variable in range(nterms-1): # why nterms-1?: intentionally excluting time, as I do not need discrepancies in time
        # Calculate time-dependent squared error
        time_diff = (extended_validation_traject[:, state_variable] - simulated_model[:, state_variable])**2
        
        # Compute cumulative RMS for divergence detection
        rms = np.sqrt(np.cumsum(time_diff) / np.arange(1, len(time_diff) + 1))
        
        # Find the first index where RMS exceeds the threshold
        if np.any(rms > threshold_for_ts):
            switch_ind = np.argmax(rms > threshold_for_ts)
        else:
            switch_ind = len(tvec)  # No significant divergence detected
        
        # Record divergence time
        tdiv[state_variable] = tvec[min(switch_ind, len(tvec) - 1)]  # Ensure bounds safety
    if print_switch_index_val:    
        print('rms was: ' + str(rms[-1]))
        print('switch_ind was: ' + str(switch_ind))

    if model_type == 'Normal DMT':
        F_ts_true = calculate_true_F_ts_DMT(sim_disp_data=full_validation_trajectory[init_cond_index:next_index], a0=a0, C1=C1, C2=C2)
        F_ts_true_sim = simulate_F_ts_from_found_model_DMT(model_to_simulate, simulated_model)
    elif model_type == 'Viscoelastic DMT':
        F_ts_true = calculate_true_F_ts_DMT_viscoelast_damp(sim_disp_data=full_validation_trajectory[init_cond_index:next_index], a0=a0, C1=C1, C2=C2, C3=C3)
        F_ts_true_sim = simulate_F_ts_from_found_model_DMT(model_to_simulate, simulated_model)

    ts_time = t_simulate[:switch_ind]

    if plot: 
        #Plots displacement:
        fig, axs = plt.subplots(1, 1, figsize=(10, 8))
        axs.plot(extended_validation_traject[:switch_ind,2], extended_validation_traject[:switch_ind,0], color = 'green', label = r'True $\eta_1$')
        axs.plot(t_simulate[:switch_ind], simulated_model[:switch_ind,0], '--', color = 'black', label = r'Found $\eta_1$')
        axs.plot(validation_trajectory_section[:,2], validation_trajectory_section[:,0], '-', color = 'red', linewidth = 3, alpha = 0.5, label = r'Cluster Extension')
        axs.set_title('Comparison between true and found dynamics', fontsize=25)
        axs.set_xlabel(r'Time [-]', fontsize=25)
        axs.set_ylabel(r'Displacement [-]', fontsize=25)
        axs.legend(loc='best', fontsize=15)
        axs.tick_params(axis='both', which='major', labelsize=25)  # Customize the font size of the tick labels
        axs.tick_params(axis='both', which='major', labelsize=25)  # Customize the font size of the tick label
        if save:
            plt.savefig(disp_plot_file_name + '.png', dpi=300, bbox_inches='tight', transparent=True)

        #Plots Velocity:
        fig, axs = plt.subplots(1, 1, figsize=(10, 8))
        axs.plot(extended_validation_traject[:switch_ind,2], extended_validation_traject[:switch_ind,1], color = 'green', label = r'True $\eta_2$')
        axs.plot(t_simulate[:switch_ind], simulated_model[:switch_ind,1], '--', color = 'black', label = r'Found $\eta_2$')
        axs.plot(validation_trajectory_section[:,2], validation_trajectory_section[:,1], '-', color = 'red', linewidth = 3, alpha = 0.5, label = r'Cluster Extension')
        axs.set_title('Comparison between true and found dynamics', fontsize=25)
        axs.set_xlabel(r'Time [-]', fontsize=25)
        axs.set_ylabel(r'Velocity [-]', fontsize=25)
        axs.legend(loc='best', fontsize=15)
        axs.tick_params(axis='both', which='major', labelsize=25)  # Customize the font size of the tick labels
        axs.tick_params(axis='both', which='major', labelsize=25)  # Customize the font size of the tick label
        if save:
            plt.savefig(vel_plot_file_name + '.png', dpi=300, bbox_inches='tight', transparent=True)

        #Checking parsimony:
        eq_fund_tab_att_pareto = convert_to_table(model_to_simulate.equations(precision=9)[1])
        eq_len_att_pareto = eq_fund_tab_att_pareto.count()

        print('')
        print('Equation with length: ' + str(eq_len_att_pareto))
        model_to_simulate.print(precision=9)
        print('')

        # Plot the force calculations:
        fig, axs = plt.subplots(1, 1, figsize=(10, 8))
        axs.plot(t_simulate, F_ts_true, color='black', label=r'True $F_{ts}$')
        axs.plot(t_simulate[:switch_ind], F_ts_true_sim[:switch_ind], '--', color='red', label=r'Found $F_{ts}$')
        axs.set_title('Comparison between true and found force', fontsize=25)
        axs.set_xlabel(r'Time [-]', fontsize=25)
        axs.set_ylabel(r'Tip-Sample Force [-]', fontsize=25)
        axs.tick_params(axis='both', which='major', labelsize=25)  # Customize the font size of the tick labels

        # Set the Y-axis to scientific notation
        formatter = ScalarFormatter(useMathText=True)
        formatter.set_scientific(True)
        formatter.set_powerlimits((-2, 2))  # Specify the limits for scientific notation
        axs.yaxis.set_major_formatter(formatter)

        if F_xlim is not None:
            axs.set_xlim([F_xlim[0], F_xlim[-1]])
        axs.legend(loc='best', fontsize=15)

        if save:
            plt.savefig(force_plot_file_name + '.png', dpi=300, bbox_inches='tight', transparent=True)

    
    return simulated_model, extended_validation_traject, F_ts_true, F_ts_true_sim

In [ ]:
def simulate_phase_space_from_most_freq_models_until_ts(model_to_simulate, validation_trajectory_section, full_validation_trajectory, solve_ivp_method, dt, t_steps_beyond_cluster, threshold_for_ts, a0, model_type, C1=None, C2=None, C3=None, disp_plot_file_name= None, vel_plot_file_name = None, force_plot_file_name= None, plot = False, save=False):
#   Version 1: This function takes a model that has been trained and simulates it in time from the cluster from which it was found until a divergence criteria is met.
#              It also shows the trajectory in the phase space 
# 
#   Parameters: model_to_simulate: Is the fit SINDy found model to simulate.
#               validation_trajectory_section: Is the array single trajectory section within "full_validation_trajectory" that we can to use as validation for the simulated SINDy found model.
#                                              Typically is a section of a cluster. Therefore, it must have a shape of (num_of_data, 3). (dist, vel, phase). Example: mult_traj_clusters_sections[traject][cluster_att][10]
#               full_validation_trajectory: Is the complete trajectory starting from transient to steady state that can be used as validation. It is used to simulate a bit furhter than the cluster size. Example: x_train_DMT_mult_traj[traject]
#               solve_ivp_method: Is a string to choose the method to perform the numerical integration like 'RK45' or 'BDF'.
#               disp_plot_file_name: Is the name for the displacement figure if Save=True.
#               force_plot_file_name: Is the name for the displacement figure if Save=True.

    t_simulate = np.arange(validation_trajectory_section[0][2], validation_trajectory_section[0][2]+(t_steps_beyond_cluster*dt), dt) #This is the reason why it only works for attractive clusters so far. t_simulate can not go further than att region "+2.5"
    init_cond = validation_trajectory_section[0]

    simulated_model = model_to_simulate.simulate(init_cond, t_simulate, integrator_kws={"method": solve_ivp_method, "rtol": 1e-5, "atol": 1e-7})

    init_cond_index = np.where(full_validation_trajectory[:,2] == t_simulate[0])[0][0] #Find where in the full trajectory the t_simulate starts for plotting and giving a score
    next_index = init_cond_index + simulated_model.shape[0] #Find where in the full trajectory the t_simulate ends or plotting and giving a score

    extended_validation_traject = np.vstack((full_validation_trajectory[init_cond_index:next_index,0], full_validation_trajectory[init_cond_index:next_index,1], full_validation_trajectory[init_cond_index:next_index,2])).T

    # Extract time vector from validation_data (assuming it's the third column)
    tvec = extended_validation_traject[:, 2]
    t_simulate = simulated_model[:,2]

    # Ensure the lengths match
    if extended_validation_traject.shape != simulated_model.shape:
        raise ValueError('Validation data and model predictions must have the same shape.' + str(extended_validation_traject.shape) + ' and ' + str(simulated_model.shape))

    tlength, nterms = simulated_model.shape  # Time length and number of variables
    abserror = np.abs(extended_validation_traject - simulated_model)  # Element-wise absolute error

    # Initialize outputs
    tdiv = np.zeros(nterms-1) 
    abserror_avg = np.zeros(nterms-1) #intentionally excluting time, as I do not need discrepancies in time
    rmse = np.zeros(nterms-1)
    abserror_over_time = np.zeros(nterms-1)

    # Loop over each state variable
    for state_variable in range(nterms-1): # why nterms-1?: intentionally excluting time, as I do not need discrepancies in time
        # Calculate time-dependent squared error
        time_diff = (extended_validation_traject[:, state_variable] - simulated_model[:, state_variable])**2
        
        # Compute cumulative RMS for divergence detection
        rms = np.sqrt(np.cumsum(time_diff) / np.arange(1, len(time_diff) + 1))
        
        # Find the first index where RMS exceeds the threshold
        if np.any(rms > threshold_for_ts):
            switch_ind = np.argmax(rms > threshold_for_ts)
        else:
            switch_ind = len(tvec)  # No significant divergence detected
        
        # Record divergence time
        tdiv[state_variable] = tvec[min(switch_ind, len(tvec) - 1)]  # Ensure bounds safety
    print('rms was: ' + str(rms[-1]))
    print('switch_ind was: ' + str(switch_ind))

    simulated_model_until_ts = np.vstack((simulated_model[:switch_ind,0], simulated_model[:switch_ind,1], simulated_model[:switch_ind,2])).T

    if model_type == 'Normal DMT':
        F_ts_true = calculate_true_F_ts_DMT(sim_disp_data=full_validation_trajectory[init_cond_index:next_index], a0=a0, C1=C1, C2=C2)
        F_ts_true_sim = simulate_F_ts_from_found_model_DMT(model_to_simulate, simulated_model)
    elif model_type == 'Viscoelastic DMT':
        F_ts_true = calculate_true_F_ts_DMT_viscoelast_damp(sim_disp_data=full_validation_trajectory[init_cond_index:next_index], a0=a0, C1=C1, C2=C2, C3=C3)
        F_ts_true_sim = simulate_F_ts_from_found_model_DMT(model_to_simulate, simulated_model)

    if plot: 
        #Plots displacement:
        fig, axs = plt.subplots(1, 1, figsize=(10, 8))
        axs.plot(full_validation_trajectory[-16000:,0], full_validation_trajectory[-16000:,1], color = 'green', label = r'True $\eta_1$')
        axs.plot(simulated_model[:switch_ind,0], simulated_model[:switch_ind,1], '--', color = 'black', linewidth = 2.5, label = r'Found $\eta_1$')
        axs.plot(validation_trajectory_section[:,0], validation_trajectory_section[:,1], '-', color = 'red', linewidth = 3, alpha = 0.5, label = r'Cluster Extension')
        axs.set_title('Comparison between true and found dynamics', fontsize=25)
        axs.set_xlabel(r'Time [-]', fontsize=25)
        axs.set_ylabel(r'Displacement [-]', fontsize=25)
        axs.legend(loc='best', fontsize=15)
        axs.tick_params(axis='both', which='major', labelsize=25)  
        axs.tick_params(axis='both', which='major', labelsize=25)  
        if save:
            plt.savefig(disp_plot_file_name + '.png', dpi=300, bbox_inches='tight', transparent=True)
    
    return simulated_model_until_ts, extended_validation_traject, F_ts_true, F_ts_true_sim

## Functions to evaluate models with AIC

In [ ]:
def calc_ts_and_RMSE_single_model_disp_vel_F_ts(validation_data, simulated_data, validation_force, simulated_force, threshold_for_ts, plot =False):
#   Version 1: This function calculates divergence time, absolute error, average absolute error, and RMSE between model predictions and validation data.
#
#   Parameters: validation_data: np.ndarray Validation data (time-series matrix of shape [time, variables]).
#               simulated_data: np.ndarray Model predictions (time-series matrix of the same shape as validation_data).

    # Ensure inputs are numpy arrays
    validation_data = np.asarray(validation_data)
    simulated_data = np.asarray(simulated_data)

    # Extract time vector from validation_data (third column)
    tvec = validation_data[:, 2]
    t_simulate = simulated_data[:,2]
    
    # Ensure the lengths match
    if validation_data.shape != simulated_data.shape:
        raise ValueError("Validation data and model predictions must have the same shape.")
    
    tlength, nterms = simulated_data.shape  # Time length and number of variables
    abserror = np.abs(validation_data - simulated_data)  # Element-wise absolute error

    # Initialize outputs
    tdiv = np.zeros(nterms-1) 
    abserror_avg = np.zeros(nterms-1) #intentionally excluting time, as discrepancies in time are not needed
    rmse = np.zeros(nterms-1)
    abserror_over_time = np.zeros(nterms-1)
    abserror_over_time_F_ts = np.zeros(1)

    # Loop over each state variable
    for state_variable in range(nterms-1): # why nterms-1?: intentionally excluting time, as I do not need discrepancies in time
        # Calculate time-dependent squared error
        time_diff = (validation_data[:, state_variable] - simulated_data[:, state_variable])**2
        
        # Compute cumulative RMS for divergence detection
        rms = np.sqrt(np.cumsum(time_diff) / np.arange(1, len(time_diff) + 1))
        
        # Find the first index where RMS exceeds the threshold
        if np.any(rms > threshold_for_ts):
            switch_ind = np.argmax(rms > threshold_for_ts)
            
        else:
            switch_ind = len(tvec)  # No significant divergence detected

        # Record divergence time
        tdiv[state_variable] = tvec[min(switch_ind, len(tvec) - 1)]  # Ensure bounds safety

        # Compute average absolute error and RMSE up to divergence point
        abserror_over_time[state_variable] = np.sum((simulated_data[:switch_ind, state_variable]-validation_data[:switch_ind, state_variable])**2)/(np.mean(tdiv))
        
        E_avg = np.sum(abserror_over_time)/(nterms-1)
        
        rmse[state_variable] = np.sqrt(np.mean(time_diff[:switch_ind]))

    abserror_over_time_F_ts = np.sum((simulated_force[:switch_ind]-validation_force[:switch_ind])**2)/(np.mean(tdiv))

    E_avg_F_ts = np.sum(abserror_over_time_F_ts)/(1)   

    if plot:
        print('The value for t_s in e1 and e2 was respectfully: ' + str(tdiv)) 
        print('Validation data time goes until ' +str(validation_data[-1,2]))
        print('Error information:')
        print('The avg absolute error in e1 and e2 was respectfully: ' + str(abserror_avg))
        print('The RMSE in e1 and e2 was respectfully: ' + str(rmse)) 

        #Plots displacement:
        fig = plt.figure()
        plt.plot(validation_data[:switch_ind,2], validation_data[:switch_ind,0], color = 'green', label = r'True $\eta_1$')
        plt.plot(t_simulate[:switch_ind], simulated_data[:switch_ind,0], '--', color = 'black', label = r'Found $\eta_1$')
        plt.title('Comparison between true dynamics and found dynamics')
        plt.xlabel('Time [-]')
        plt.ylabel('Displacement [-]')
        plt.legend(loc='best')

        #Plots Velocity:
        fig = plt.figure()
        plt.plot(validation_data[:switch_ind,2], validation_data[:switch_ind,1], color = 'green', label = r'True $\eta_2$')
        plt.plot(t_simulate[:switch_ind], simulated_data[:switch_ind,1], '--', color = 'black', label = r'Found $\eta_2$')
        plt.title('Comparison between true dynamics and found dynamics')
        plt.xlabel('Time [-]')
        plt.ylabel('Velocity [-]')
        plt.legend(loc='best')

    return tdiv, abserror, E_avg, E_avg_F_ts, rmse

In [ ]:
def calculate_AIC_single_model_disp_vel_F_ts(E_avg, E_avg_F_ts, analyzed_model, N):
#   Version 1: This function calculates Akaike Information Criterion (AIC), corrected AIC (AICc), and Bayesian Information Criterion (BIC) for a given model.
#
#   Parameters: E_avg: np.ndarray Vector of absolute errors between the model predictions and the validation data.
#               E_avg_F_ts: np.ndarray Vector of absolute errors between the model predictions and the validation data for the force.
#               analyzed_model: Model that produced the data that was used to calculate the abserror.
#               N: Number of data points in the validation set.
    
    #Checking parsimony:
    analyzed_eq_tab = convert_to_table(analyzed_model.equations(precision=9)[1])
    p = analyzed_eq_tab.count()

    if p == 0:
        p = 1  

    # Calculate the mean squared error (MSE)
    mse = np.mean(E_avg**2)

    # Log-likelihood (assuming normally distributed errors)
    logL = -N * np.log(mse) / 2

    # Akaike Information Criterion (AIC)
    epsilon = 1e-12
    aic = (2*p) + (N*np.log((E_avg + epsilon)/N)) + (N*np.log((E_avg_F_ts + epsilon)/N))

    # Corrected AIC for small datasets    
    aic_c = aic + (2 * p * (p + 1)) / (N - p - 1)

    # Bayesian Information Criterion (BIC)
    bic = -2 * logL + p * np.log(N)

    # Store results in a dictionary
    IC = {
        'aic': aic,
        'aic_c': aic_c,
        'bic': bic
    }

    return IC

In [ ]:
def simulate_cluster_get_E_avg_until_ts_n_calculate_AIC_unique_models(unique_models_df, validation_trajectory_section, full_validation_trajectory, t_steps_beyond_cluster, threshold_for_ts, solve_ivp_method, dt, a0, model_type, C1=None, C2=None, C3=None):

    AIC_models_in_cluster = []

    for model_to_study in tqdm(unique_models_df['Candidate Model'], desc="Processing models", unit="model"):

        simulated_data, extended_val_trajec, F_ts_true, F_ts_sim = simulate_results_from_cluster(model_to_simulate = model_to_study,
                                                    validation_trajectory_section = validation_trajectory_section,
                                                    full_validation_trajectory = full_validation_trajectory,
                                                    solve_ivp_method = solve_ivp_method, dt=dt, t_steps_beyond_cluster = t_steps_beyond_cluster,
                                                    model_type=model_type, a0=a0, C1=C1, C2=C2, C3=C3, 
                                                    plot = False)

        tdiv, abserror, abserror_avg, E_avg_F_ts, rmse = calc_ts_and_RMSE_single_model_disp_vel_F_ts(validation_data = extended_val_trajec,
                                                    simulated_data = simulated_data, validation_force = F_ts_true, 
                                                    simulated_force = F_ts_sim, threshold_for_ts = threshold_for_ts,
                                                    plot = False)

        AIC = calculate_AIC_single_model_disp_vel_F_ts(E_avg = abserror_avg, E_avg_F_ts = E_avg_F_ts, analyzed_model = model_to_study, N = 1)
        AIC_models_in_cluster.append(AIC)
    unique_models_df['AIC_c'] = AIC_models_in_cluster

    simulated_information = [simulated_data, extended_val_trajec, F_ts_true, F_ts_sim]
    error_information = [tdiv, abserror, abserror_avg, E_avg_F_ts, rmse]

    return simulated_information, error_information, AIC_models_in_cluster

In [ ]:
def get_relative_AIC_c_score(AIC_score_models_in_cluster):
    
    min_AIC_c = np.inf
    relative_AIC_c_list = []
    for model in AIC_score_models_in_cluster:
        if model['aic_c'] < min_AIC_c:
            min_AIC_c = model['aic_c']
    
    for model in AIC_score_models_in_cluster:
        rel_AIC_c = model['aic_c'] - min_AIC_c
        relative_AIC_c_list.append(rel_AIC_c)
        
    return relative_AIC_c_list 

## Functions for statistical analysis of AIC

In [ ]:
def extract_active_cand_functions_from_found_eq(candidate_model):

    # Get the feature names
    candidate_functions = candidate_model.feature_library.get_feature_names()

    # Get the coefficient matrix (Theta)
    coefficients = candidate_model.coefficients()

    active_features = []
    for eq_coeffs in coefficients:
        active_features.append([candidate_functions[i] for i in range(len(eq_coeffs)) if eq_coeffs[i] != 0])

    return active_features[1] # to extract only features from e2' equation

## Functions to find Intermolecular distance value

In [ ]:
def find_intermolecular_distance(found_SINDy_model, test_trajectory, x_train_synthetic_mult_traj, analyzed_cluster, init_cluster_size, error_threshold, theoretical_a0, solve_ivp_method, image_name=None, plot_results = False, save=False):

    x_test_regime_mult_traj = []
    init_cond_to_simulate = []
    a0_temp_list = []
    error_list = []
    error_temp = 1e-7

    while error_temp < error_threshold: #1e-3
        x_test_regime_mult_traj.clear()
        init_cond_to_simulate.clear()
        for i in analyzed_cluster:#[0]:
            x_test_traject_temp = np.vstack((test_trajectory[i:(i+init_cluster_size),0], test_trajectory[i:(i+init_cluster_size),1], test_trajectory[i:(i+init_cluster_size),2])).T
            x_test_regime_mult_traj.append(x_test_traject_temp)
            init_cond_to_simulate.append(test_trajectory[i])
        
        found_SINDy_model_simulation = found_SINDy_model.simulate(init_cond_to_simulate[0], x_test_regime_mult_traj[0][:,2], integrator_kws={"method": solve_ivp_method, "rtol": 1e-5, "atol": 1e-7})
        error_temp = (((found_SINDy_model_simulation[:,0][-1])-(x_test_regime_mult_traj[0][:,0][-1]))**2)+1*(((found_SINDy_model_simulation[:,1][-1])-(x_test_regime_mult_traj[0][:,1][-1]))**2)
        error_list.append(error_temp)
        a0_temp_list.append(x_test_regime_mult_traj[0][:,0][-1])
        a0_temp = x_test_regime_mult_traj[0][:,0][-1]
        init_cluster_size=init_cluster_size+1
    
    a0_estimation_error = abs(round(((a0_temp-(1-theoretical_a0))/(a0_temp))*100, 9))

    if plot_results:
        fig, axs = plt.subplots(1, 1, figsize=(10, 8))
        axs.plot(x_train_synthetic_mult_traj[0][:,0], x_train_synthetic_mult_traj[0][:,1], color='black', label='Test data', alpha=0.5, linewidth=1.8)
        axs.plot(found_SINDy_model_simulation[:,0], found_SINDy_model_simulation[:,1], color='red', linestyle = '-', label='Simualted data') 
        axs.plot(x_test_regime_mult_traj[0][:,0], x_test_regime_mult_traj[0][:,1], color='blue', linestyle = '--', label='test data')
        axs.set_xlabel(r'$\bar{\eta}_1$ (-)', fontsize=25)
        axs.set_ylabel(r'$\bar{\eta}_2$ (-)', fontsize=25)
        axs.set_title('Phase Space of DMT Model: Evolution of \nUsing  Simulated data from a repulsive Cluster', fontsize=25)
        axs.legend(loc='upper left', fontsize=15)
        axs.tick_params(axis='both', which='major', labelsize=25)  # Customize the font size of the tick labels
        axs.tick_params(axis='both', which='major', labelsize=25)  # Customize the font size of the tick label
        if save:
            plt.savefig(image_name + '.png', dpi=300, bbox_inches='tight', transparent=True)

    print('a0 = ' +str(1-theoretical_a0))
    print('Found a0 = ' +str(round(a0_temp, 7)))
    print('The error in estimating a0 is: ' + str(abs(round(((a0_temp-(1-theoretical_a0))/(a0_temp))*100, 4))) + str(' %'))

    return a0_temp, a0_temp_list, a0_estimation_error, error_list, found_SINDy_model_simulation, x_test_regime_mult_traj

## Functions for plotting equation bar charts

In [ ]:
def normalize_n_compare_sindy_found_coefficients(original_coeffs, identified_coeffs):
#   Version 1: Normalize identified coefficients by comparing to original ones
#              - For true terms: ratio of identified to original.
#              - For false positives: relative to largest true coefficient.
#
#   Parameters: original_coeffs: np.ndarray Array of coeeficients in the original dyanmics.
#               identified_coeffs: np.ndarray Array of coeeficients in the found dyanmics.
#
#   Returns: Normalized array same shape as identified_coeffs

    true_mask = original_coeffs != 0
    true_nonzero_values = np.abs(original_coeffs[true_mask])
    if true_nonzero_values.size == 0:
        raise ValueError("No non-zero coefficients in original model.")
    max_true_coeff = np.max(true_nonzero_values)

    normalized = np.zeros_like(identified_coeffs)

    for i in range(original_coeffs.shape[0]):
        for j in range(original_coeffs.shape[1]):
            if true_mask[i, j]:
                # Avoid division by zero (shouldn’t happen if mask is correct)
                if original_coeffs[i, j] != 0:
                    normalized[i, j] = np.abs(identified_coeffs[i, j]) / np.abs(original_coeffs[i, j])
            else:
                normalized[i, j] = np.abs(identified_coeffs[i, j]) / max_true_coeff

    return normalized

np.random.seed(100)
def plot_dynamics_bars(original_coeffs, identified_coeffs, plot_file_name, n_terms_correction=None, save_plot=False):
    n_eqs, n_terms = original_coeffs.shape
    normalized_identified = normalize_n_compare_sindy_found_coefficients(original_coeffs, identified_coeffs)

    candidate_labels = [f"$\\theta_{{{i+1}}}$" for i in range(identified_coeffs.shape[1])]

    fig, axs = plt.subplots(n_eqs, 1, figsize=(18, 2.4 * n_eqs), sharex=True)

    if n_eqs == 1:
        axs = [axs]

    bar_width = 0.35
    x = np.arange(n_terms)
    
    for i in range(n_eqs):
        axs[i].bar(x - bar_width/2, (original_coeffs[i] != 0).astype(float), width=bar_width, label='Original Dynamics', color='black')
        axs[i].bar(x + bar_width/2, normalized_identified[i], width=bar_width, label='Identified Dynamics', color='mediumseagreen')
        axs[i].set_ylim(0, 1.1)
        # axs[i].set_ylabel(f'$\dot{{\eta}}_{i+1}$' if i < 2 else '$\dot{\phi}$', fontsize=16)
        
        axs[i].spines['right'].set_visible(False)
        axs[i].spines['top'].set_visible(False)
        
        # Increase axis line thickness
        for spine in ['left', 'bottom']:
            axs[i].spines[spine].set_linewidth(3)

        axs[i].xaxis.set_major_locator(MaxNLocator(nbins=3)) 
        axs[i].set_yticks([0.33, 0.66, 1.0])

        # Format y-axis numbers to one decimal place
        axs[i].yaxis.set_major_formatter(FormatStrFormatter('%.1f'))

        axs[i].tick_params(axis='both', which='major', labelsize=29, width=2, length=6)

        axs[i].grid(axis='y', linestyle='-', linewidth=1.0, color='gray', alpha=0.4)


    if candidate_labels is None:
        candidate_labels = [f"$\\theta_{{{i+1}}}$" for i in range(n_terms)]

    axs[-1].set_xticks(x)
    axs[-1].set_xticklabels(candidate_labels, rotation=0, ha='center', fontsize=35)
    axs[-1].set_xlim(-1.0, (n_terms-n_terms_correction) - 0.5)

    plt.tight_layout()

    if save_plot:
        fig.savefig(f'{plot_file_name}.png', transparent=True, dpi=300, bbox_inches='tight')
    plt.show()

## Accuracy calculations

In [ ]:
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

def evaluate_predictions(true_values, predicted_values, label="Variable"):
    mse = mean_squared_error(true_values, predicted_values)
    rmse = np.sqrt(mse)
    r2 = r2_score(true_values, predicted_values)
    
    # Add normalized RMSE
    value_range = np.max(true_values) - np.min(true_values)
    nrmse = rmse / value_range if value_range != 0 else np.nan  # Avoid division by zero
    
    print(f"--- {label} ---")
    print(f"MSE:    {mse:.6f}")
    print(f"RMSE:   {rmse:.6f}")
    print(f"NRMSE:  {nrmse:.4%}")
    print(f"R²:     {r2:.6f}\n")